In [ ]:
!ls data/

In [1]:
import gzip

import pandas as pd

import numpy as np
import scipy.sparse as sp

from tqdm import tqdm_notebook as tqdm

In [51]:
import iotools

In [36]:
from collections import namedtuple
Line = namedtuple('Line', ['id', 'f0', 'f1', 'idx', 'val'])
LabeledLine = namedtuple('LabeledLine', ['id', 'f0', 'f1', 'idx', 'val', 'propensity', 'label'])

def parse_features(s):
    split = s.split(' ')
    f0 = split[0]
    assert f0.startswith('0:')
    f0 = int(f0[2:])

    f1 = split[1]
    assert f1.startswith('1:')
    f1 = int(f1[2:])

    idx = []
    values = []
    
    for fv in split[2:]:
        f, v = fv.split(':')
        idx.append(int(f) - 2)
        values.append(int(v))

    return f0, f1, idx, values

def read_train(fname):
    if fname.endswith('.gz'):
        f = gzip.open(fname, 'r')
        f = map(bytes.decode, f)
    else:
        f = open(fname, 'r')

    for line in f:
        split = line.split('|')
        id = int(split[0].strip())

        label = None
        propensity = None
        features = None

        if len(split) == 4:
            l = split[1]
            assert l.startswith('l')

            l = l.lstrip('l ').strip()
            if l == '0.999':
                label = 0
            elif l == '0.001':
                label = 1
            else:
                raise Exception('ololo')

            p = split[2]
            assert p.startswith('p')
            p = p.lstrip('p ').strip()
            propensity = float(p)

            features = split[3].lstrip('f ').strip()

            f0, f1, idx, val = parse_features(features)
            yield LabeledLine(id, f0, f1, idx, val, propensity, label)
        elif len(split) == 2:
            pass

In [188]:
def conv_to_bin(src, dest, train=True):
    it = read_train(src)
    sout = open(dest, 'wb')
    dout = iotools.DataOutputStream(sout)

    for line in tqdm(it):
        dout.write_int(line.id)
        dout.write_int(line.f0)
        dout.write_int(line.f1)

        n = len(line.idx)
        dout.write_int(n)

        for i in line.idx:
            dout.write_int(i)

        for v in line.val:
            dout.write_byte(v)

        if train:
            dout.write_float(line.propensity)
            dout.write_byte(line.label)

    sout.flush()
    sout.close()

In [189]:
conv_to_bin('./data/train_0.txt', './data/train_0.bin', train=True)

ValueError: invalid literal for int() with base 10: ':1 666:2 753:1 769:1 945:2 977:1 981:4 982:1 984:1 1043:1 1351:1 1914:2 1915:2 2123:2 2355:2 2648:1 3219:1 5964:2'

133219it [00:30, 4325.97it/s]

In [182]:
def read_bin(fname):
    sin = open(fname, 'rb')
    din = iotools.DataInputStream(sin)

    while sin.read(1) != b'':
        sin.seek(-1, 1)

        id = din.read_int()
        f0 = din.read_int()
        f1 = din.read_int()
        n = din.read_int()

        idx = np.zeros(n, dtype=np.uint32)
        val = np.zeros(n, dtype=np.uint8)
        
        for i in range(n):
            idx[i] = din.read_int()

        for i in range(n):
            val[i] = din.read_byte()

        propensity = din.read_float()
        label = din.read_byte()
        
        line = LabeledLine(id, f0, f1, idx, val, propensity, label)
        yield line

    sin.close()

In [186]:
it = read_bin('data/train_0.bin')